# Final Exam Solution

## FINM 37500 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 150 points.
* You have 180 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.
Final Exam

Submission
* You will upload your solution to the Final Exam assignment on Canvas, where you downloaded this.
* Your submission should be readable, (the graders can understand your answers,) and it should include all code used in your analysis in a file format that the code can be executed. (ie. .ipynb preferred, .pdf is unacceptable.)

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation, (only for this exam.)

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `exam_data_2023-05-05.xlsx`

This file has sheets for...
* curve data - discount factors and forward volatilities
* BDT tree of rates (continuously compounded, as usual)

If useful, the following code loads the two sheets:

In [1]:
# FILEIN = '../data/exam_data_2023-05-05.xlsx'
# sheet_curves = 'rate curves'
# sheet_tree = 'rate tree'

# curves = pd.read_excel(FILEIN, sheet_name=sheet_curves).set_index('tenor')
# ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
# ratetree.columns.name = 'time'

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 60     |
| 2       | 30     |
| 3       | 20     |
| 4       | 15     |
| 5       | 15     |
| 6       | 10     |

### Each numbered question is worth 5pts.

# 1. Short Answer

### No Data Needed

These problem does not require any data file. Rather, analyze the situation conceptually, based on the information below. 

A few questions may require simple arithmetic.

## 1.1

Consider a future on a 5-year treasury. If the exchange changes the treasury futures conversion factor to use a 3\% discount instead of 6\%, would you expect substantial changes to the...
* the futures price?
* the forward price of the underlying CTD treasury?

Explain.

**ANSWER:**  The future price would substanitally change

## 1.2

Explain how SABR allows for an extra "channel" of delta hedging.

## 1.3

Local volatility fits quoted volatilities exactly, whereas SABR does not.

What advantage does SABR have over local volatility? How does it relate to the fact above?

## 1.4
Relative to three years ago, do you think BDT or Ho-Lee is better suited to today's interest rates?

Explain.

## 1.5
True or False: To value a caplet in Black's formula, we input the spot interest rate as the underlying.

Explain.

## 1.6
Should we expect a delta-hedged callable bond to underperform or overperform a delta-hedged vanilla bond? Explain.

## 1.7
True or False: In a frictionless market, gross basis and net basis are nonzero before maturity, but they converge to zero for the CTD bond at maturity.

## 1.8
True or False: Bonds with higher coupons have larger "forward drops".

## 1.9
True or False: We expect that high duration bonds are typically CTD.

## 1.10

True or False: If forward volatility is decreasing over maturity then flat volatility is also decreasing over maturity.

## 1.11

True or False: From fed funds futures prices, we can extract the probability of interest rate movements for any meeting date.

## 1.12

How is Black's model useful---and necessary--- in fixed income derivatives, even if we choose to use a more complex, proprietary model.

***

In [1]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime

import sys
sys.path.insert(0, '../cmds')
from ficcvol import *
from binomial import *
from ratecurves import *

In [2]:
import datetime
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

import matplotlib.ticker as mtick
from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)

In [3]:
FILEIN = '../data/exam_data_2023-05-05.xlsx'
sheet_curves = 'rate curves'
sheet_tree = 'rate tree'

curves = pd.read_excel(FILEIN, sheet_name=sheet_curves).set_index('tenor')
ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

# 2. Bond Pricing

## 2.1
Use the BDT tree to price a bond.
* Time-to-maturity is 3 years
* Coupon rate is 1\%.

As usual, 
* coupons are paid semiannually.
* face value is $N=100$.

Display the tree of the bond's value, where each node is **as usual** the ex-coupon dirty price. 
* Thus, the value at node 0.5 is just after the coupon was paid.
* The value at node 0.25 includes accrued interest.

In [9]:
T = 3
cpn = 0.01
freq = 2
N = 100

In [5]:
ratetree

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,0.0504,0.056948,0.061086,0.064897,0.064964,0.071312,0.085580,0.103667,0.121216,0.160705,0.210246,0.277823,0.356583
1,NaN,0.041512,0.044528,0.047306,0.047355,0.051982,0.062383,0.075567,0.088359,0.117144,0.153257,0.202517,0.259928
2,NaN,NaN,0.030691,0.032606,0.032640,0.035829,0.042997,0.052085,0.060902,0.080742,0.105633,0.139586,0.179156
3,NaN,NaN,NaN,0.021250,0.021271,0.023350,0.028022,0.033944,0.039690,0.052620,0.068842,0.090968,0.116757
4,NaN,NaN,NaN,NaN,0.010906,0.011971,0.014367,0.017403,0.020349,0.026978,0.035295,0.046639,0.059861
5,NaN,NaN,NaN,NaN,NaN,0.005955,0.007146,0.008657,0.010122,0.013420,0.017557,0.023200,0.029777
6,NaN,NaN,NaN,NaN,NaN,NaN,0.003589,0.004347,0.005083,0.006739,0.008816,0.011650,0.014952
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002247,0.002627,0.003483,0.004557,0.006022,0.007729
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001424,0.001888,0.002470,0.003264,0.004190


In [7]:
FREQTREE = 4
dt = 1/ FREQTREE

In [12]:
payoff = lambda r: payoff_bond(r,dt)
tsteps = int(T / dt)
ratetree = ratetree.iloc[:tsteps,:tsteps]
ratetree

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75
state,,,,,,,,,,,,
0,0.0504,0.056948,0.061086,0.064897,0.064964,0.071312,0.085580,0.103667,0.121216,0.160705,0.210246,0.277823
1,NaN,0.041512,0.044528,0.047306,0.047355,0.051982,0.062383,0.075567,0.088359,0.117144,0.153257,0.202517
2,NaN,NaN,0.030691,0.032606,0.032640,0.035829,0.042997,0.052085,0.060902,0.080742,0.105633,0.139586
3,NaN,NaN,NaN,0.021250,0.021271,0.023350,0.028022,0.033944,0.039690,0.052620,0.068842,0.090968
4,NaN,NaN,NaN,NaN,0.010906,0.011971,0.014367,0.017403,0.020349,0.026978,0.035295,0.046639
5,NaN,NaN,NaN,NaN,NaN,0.005955,0.007146,0.008657,0.010122,0.013420,0.017557,0.023200
6,NaN,NaN,NaN,NaN,NaN,NaN,0.003589,0.004347,0.005083,0.006739,0.008816,0.011650
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002247,0.002627,0.003483,0.004557,0.006022
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001424,0.001888,0.002470,0.003264


In [16]:
cftree = construct_bond_cftree(T, FREQTREE, cpn)
cftree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
1,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
2,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
3,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
4,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
5,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
6,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
7,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0
8,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,0.0


In [19]:
bintree_pricing(payoff, ratetree, cftree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,90.695948,90.180201,89.762006,89.406697,89.131659,88.870492,88.802082,89.167874,90.136825,91.797167,94.879589,100.0
1,NaN,93.511684,93.184558,92.879945,92.606526,92.311655,92.136089,92.277061,92.881235,94.023102,96.241038,100.0
2,NaN,NaN,95.789838,95.575419,95.363313,95.107130,94.902168,94.891490,95.192599,95.888482,97.393735,100.0
3,NaN,NaN,NaN,97.479856,97.352064,97.182181,97.023546,96.964128,97.077760,97.417603,98.293687,100.0
4,NaN,NaN,NaN,NaN,98.646104,98.560115,98.478734,98.447114,98.503178,98.674015,99.121507,100.0
5,NaN,NaN,NaN,NaN,NaN,99.270737,99.232335,99.219030,99.249558,99.337115,99.562045,100.0
6,NaN,NaN,NaN,NaN,NaN,NaN,99.604935,99.600531,99.618424,99.664947,99.779842,100.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.788137,99.799236,99.825234,99.886140,100.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.889178,99.904383,99.938258,100.0


## 2.2

Display the tree of the bond's **clean price** (still ex-coupon, as usual.)

## 2.3

Consider a European **put** option on the bond.
* Expiration of the option is 1 year.
* Strike is equal to face value, $100$.
* At exercise, the option holder receives the strike plus accrued interest.

Display the tree of the option price.

## 2.4

Consider the same option, but this time **American**.

Display the tree of the option price.

## 2.5

Is there a node where the option is in-the-money, yet it is not optimal to exercise? 

Briefly explain, and if so, give an example.

## 2.6 

Consider an **amortizing bond** with 
* maturity 2.5 years.
* coupon rate of 40\%.
* Notional of \$100.

Unlike a typical bond, it pays a coupon semiannually, but it does not ever pay the face value. In that sense, one could view the coupons as amortized payments of the face value.

Value the amortizing bond and display the valuation tree.

***

# 3 Floorlet

## 3.1 (5pts)

Consider a **floorlet** with
* Strike is $0.06$
* Notional $N=100$

As usual, the floorlet has a one-quarter lag between when the payoff is determined and when it is received.
* determined at $T=2$
* payoff at $T=2.25$.

Use the BDT tree of rates to value the floorlet. Display the valuation tree.

In [20]:
K = 0.06
N = 100
freq = 4
tstart = 2
T = tstart + 1 / freq


## 3.2 (5pts)

Regardless of your price above, assume the market is quoting the floorlet at a price of \$0.80.

Use Black's formula to calculate the implied volatility of this floorlet, supposing the underlying is currently 3.5\%.

## 3.3 (5pts)

Based on your answers to 3.1 and 3.2, is the OAS positive or negative? Explain.

## 3.4 (5pts)

Consider a floorlet which pays immediately upon the payoff being determined rather than having the one-quarter lag.

Price this alternate floorlet and display the valuation tree.

***

# 4. Swaps

## 4.1 

Consider a **paying-fixed** swap which resets (and make payments) **quarterly**.
* Maturity is $T=3$
* Notional is $N=100$.
* Swap rate is 3.53\%.
* The floating rate is the quarterly compounded interest rate.

(Recall that paying-fixed means you pay the swap rate and receive the floating rate.)

Display the cashflow tree.

## 4.2

Display the value tree.

## 4.3

Consider a **european** swaption, (option to enter into the paying-fixed swap).
* expiration is $T=1$
* there is no strike at exercise.
    - could think of this as a strike of 0
    - but often referred to as a strike of 3.53\%, the fixed rate on the underlying swap
    
Display the swaption valuation tree.

***

# 5. Treasury Futures

## 5.1

Consider a bond with
* Market (clean) price of 97.50
* Futures conversion ratio of 0.95
* Futures price is 103.

It has
* Coupon rate of 1\% 
* face value of 100.

As for the coupon timing,
* The bond paid a coupon 38 days ago.
* It will pay another coupon in 144 days.
* The repo rate over the next 90 days is (annualized in the usual way) 5\%.

Calculate the forward price of the bond for delivery in 90 days. 

## 5.2

Calculate the gross and net basis of the bond. The futures price is 103.

## 5.3

*Unrelated to 5.1 and 5.2...*

Suppose you are pricing a futures contract on the 5-year treasury. The contract expires in 1 year, $T=1$.

Based on the deliverable bonds, we find the CTD prices at expiration as listed in the table for each state.

| state |CTD Converted Clean Price at $T=1$ |
|--------|--------|
|0   | 99.05|
|1       |102.47 |
|2       | 105.31|
|3       |107.62 |
| 4      |109.79|

Using the table, calculate the price of the futures contract.

*Note that there is no need to build trees of the underlying bonds and figure out the CTD for each state. All that has been done, and you have the table above.*

***

# 6. Fitting the BDT Tree

Fit a BDT Tree. We will not use it for any other questions on the exam. 


## 6.1
Use the discount rates provided in the data file.

Instead of using the forward volatilities, use a constant $\sigma=0.50$ at every step.

Report the estimated thetas.

## 6.2
Regardless of your previous answer, suppose your estimated thetas are all equal to 0.5.

Display the interest-rate tree.

***